In [1]:
# check token connection
from dotenv import load_dotenv
load_dotenv(override=True) # check api keys

True

In [3]:
from langchain.chat_models import init_chat_model # LLM model
from langgraph.prebuilt import create_react_agent # to create an agent
from langchain_core.tools import tool # for tools
from langgraph.checkpoint.memory import MemorySaver # to handle memory threads
from uuid import uuid4 # to create a memory thread id
import ast # to create syntaxt tree from a string mathematical expression
import operator # import basic math operators as functions

In [4]:
import enum # to create a fixed list of products

class ProductNames(enum.Enum):
    RedShirt = "red shirt"
    BlueJeans = "blue jeans"
    BlackShoes = "black shoes"
    WhiteSocks = "white socks"
    GrayHat = "gray hat"
    GreenScarf = "green scarf"
    YellowBelt = "yellow belt"
    PurpleGloves = "purple gloves"
    BrownJacket = "brown jacket"
    PinkSweater = "pink sweater"

# list of products and info

PRODUCTS = [
    {
        "name": ProductNames.RedShirt.value,
        "price": 20.00,
        "discount": 0.10,
        "currency": "EUR",
        "description": "A stylish red shirt made from high-quality cotton.",
    },
    {
        "name": ProductNames.BlueJeans.value,
        "price": 40.00,
        "discount": 0.20,
        "currency": "EUR",
        "description": "A pair of comfortable blue jeans made from durable denim.",
    },
    {
        "name": ProductNames.BlackShoes.value,
        "price": 30.00,
        "discount": 0.15,
        "currency": "EUR",
        "description": "A pair of elegant black shoes made from genuine leather.",
    },
    {
        "name": ProductNames.WhiteSocks.value,
        "price": 5.00,
        "discount": 0.05,
        "currency": "EUR",
        "description": "A pack of soft white socks made from breathable cotton.",
    },
    {
        "name": ProductNames.GrayHat.value,
        "price": 10.00,
        "discount": 0.10,
        "currency": "EUR",
        "description": "A stylish gray hat made from lightweight fabric.",
    },
    {
        "name": ProductNames.GreenScarf.value,
        "price": 15.00,
        "discount": 0.12,
        "currency": "EUR",
        "description": "A warm green scarf made from wool.",
    },
    {
        "name": ProductNames.YellowBelt.value,
        "price": 8.00,
        "discount": 0.08,
        "currency": "EUR",
        "description": "A yellow belt made from synthetic leather.",
    },
    {
        "name": ProductNames.PurpleGloves.value,
        "price": 12.00,
        "discount": 0.10,
        "currency": "EUR",
        "description": "A pair of purple gloves made from soft fabric.",
    },
    {
        "name": ProductNames.BrownJacket.value,
        "price": 50.00,
        "discount": 0.25,
        "currency": "EUR",
        "description": "A brown jacket made from genuine leather.",
    },
    {
        "name": ProductNames.PinkSweater.value,
        "price": 25.00,
        "discount": 0.15,
        "currency": "EUR",
        "description": "A pink sweater made from high-quality cotton.",
    }
]

# maps syntax operations to corresponding operators
OP_MAP = {
    # binary
    ast.Add: operator.__add__,
    ast.Sub: operator.__sub__,
    ast.Div: operator.__truediv__,
    ast.Mult: operator.__mul__,
    # unary
    ast.UAdd: operator.__pos__,
    ast.USub: operator.__neg__,
}

# establish allowed types in the syntax tree
ALLOWED_TYPES = {int, float}

# read the syntaxt tree in a recursive way and compute expressions
def compute(expr):
    match expr:
        case ast.Constant(value=val):
            if type(val) not in ALLOWED_TYPES:
                raise SyntaxError(
                    f"Not a number {val!r}"
                )
            return val
        case ast.UnaryOp(op=opr, operand=val):
            try:
                return OP_MAP[type(opr)](compute(val))
            except KeyError:
                raise SyntaxError(f"Unknown operation {ast.unparse(expr)}")
        case ast.BinOp(op=opr, left=lft, right=rgt):
            try:
                return OP_MAP[type(opr)](compute(lft), compute(rgt))
            except KeyError:
                raise SyntaxError(f"Unknown operation {ast.unparse(expr)}")

In [5]:
# tools definition

@tool
def get_product_info(product_name: ProductNames) -> dict:
    """
    - this tool returns the identifier for a given product
    - the input is the name of a product 
    - the format of the input is a member of ProductNames
    """
    for product in PRODUCTS:
        if product["name"] == product_name.value:
            return product
    return None

@tool
def calculator(expression: str) -> float:
    """
    - this tool resturns the value of a simple mathematical expression
    - the input is a string with a mathematical expression (for instance, '(-2+3)*4 + 10')
    - the format of the input is a string
    """
    tree = ast.parse(expression,mode='eval')
    return compute(tree.body)

In [6]:
# define LLM model
llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [7]:
# custom prompt, memory thread and agent definition

SHOPPING_AGENT_PROMPT = """"
# you are an efficient customer supporter from a prestigiuos and refined shop
# your task is either to give information about products or give the overall price of the required products (considering the discount). 
# if you are asked to give the price, it is up to you to build the mathematical expression. Use the calculator tool to give the price.
# for example, the mathematical expression for the price of two red shirts and one yellow belt becomes the following:
# '2*(20.00 - 0.10*20.00) + ( 8.00 - 0.08*8.00 )'
# and the final price is 43.36 EUR. 
# if that product is not in the PRODUCT list, tell the customer that you do not sell that item
"""

memory = MemorySaver()

graph_with_memory = create_react_agent(
    model=llm,
    tools=[get_product_info,calculator],
    prompt=SHOPPING_AGENT_PROMPT,
    checkpointer=memory,
)

my_config = {
    "configurable": {
        "thread_id": str(uuid4()) # The thread_id is a unique identifier for each conversation thread.
        }
}


In [8]:
question = """"How much does the red shirt cost?"""
response = graph_with_memory.invoke({"messages": [{"role": "user", "content": question}]},config=my_config)
ai_messages = [messages for messages in response['messages'] if messages.type =='ai']
print(ai_messages[-1].content)

The red shirt costs 20.00 EUR. We are currently offering a 10% discount on this item.


In [9]:
question = """"so, how much should I pay?"""
response = graph_with_memory.invoke({"messages": [{"role": "user", "content": question}]},config=my_config)
ai_messages = [messages for messages in response['messages'] if messages.type =='ai']
print(ai_messages[-1].content)

The red shirt costs 20.00 EUR, but with the 10% discount, you only pay 18.00 EUR.


In [10]:
question = """"I also need a pair of blue jeans. How much is the total?"""
response = graph_with_memory.invoke({"messages": [{"role": "user", "content": question}]},config=my_config)
ai_messages = [messages for messages in response['messages'] if messages.type =='ai']
print(ai_messages[-1].content)

The blue jeans cost 40.00 EUR. With a 20% discount, the price is 32.00 EUR. The red shirt is 18.00 EUR with the discount. Therefore, the total cost would be 50.00 EUR.
